![Unidata organization logo](https://github.com/Unidata/metpy-ams-2023/raw/main/logos/unidata_logo_horizontal.png)

# Surface Data

> Surface observations might recall scary memories of METARs, but calm your fears! We can continue to rely on the good work of our community and the powerful tools in this space. We can get straight to dataframes and in-python station model plots without ever having to decode a METAR by hand. (But don't worry, that option is on the table for you still!)

*MetPy for your Data: Analyzing Meteorological Observations in Python*  
*2023 AMS Annual Meeting*

## Table of Contents

* [1: Pandas and point data](#dataread)
* [2: Station Plots for visualizing observations](#stationplot)
* [3: A time series of observations](#timeseries)

<a name="dataread"></a>
## 1. Using Pandas to get point data from Iowa State ASOS Archive

https://mesonet.agron.iastate.edu/request/download.phtml

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

# Set a date and time for the observations you want
date = datetime.utcnow()

# Set a time window to look for observations
dt = timedelta(minutes=25)

# Use datetime and time window to set final start/end times
sdate = date - dt
edate = date + dt

Construct data access URL using start and end date/time.

In [ ]:
data_url = ('http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&'
            f'year1={sdate.year}&month1={sdate.month}&day1={sdate.day}&hour1={sdate.hour}&minute1={sdate.minute}&'
            f'year2={edate.year}&month2={edate.month}&day2={edate.day}&hour2={edate.hour}&minute2={edate.minute}')

Request Data

In [ ]:
# Use Pandas remote csv reading capability to grab desired data
# Make Missing values ('M') into NaNs
# Replace Trace precip with very small float value
df = pd.read_csv(data_url, skiprows=5, na_values=['M'], low_memory=False).replace('T', 0.00001)

In [ ]:
# Since we may have gotten more than 1 ob in our time window, only keep the latest observation
data = df.groupby('station').tail(1)

Let's take a look at what data we got!

In [ ]:
data

Thta's a lot of data - What are all of the variables that we have available?

In [ ]:
# Use our column names to list the available variables
data.columns.values

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="stationplot"></a>
## 2. Making a station plot
 * Make data request
 * Request data closest to a specific time

In [ ]:
# Set a date and time for the observations you want
date = datetime(1993, 3, 13, 12)

# Set a time window to look for observations
dt = timedelta(minutes=15)

# Use datetime and time window to set final start/end times
sdate = date - dt
edate = date + dt

# Use Pandas remote csv reading capability to grab desired data
# Make Missing values ('M') into NaNs
# Replace Trace precip with very small float value
df = pd.read_csv('http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&'
                 'tz=Etc/UTC&format=comma&latlon=yes&elev=yes&'
                 f'year1={sdate.year}&month1={sdate.month}&day1={sdate.day}&hour1={sdate.hour}&minute1={sdate.minute}&'
                 f'year2={edate.year}&month2={edate.month}&day2={edate.day}&hour2={edate.hour}&minute2={edate.minute}',
                 skiprows=5, na_values=['M'], low_memory=False).replace('T', 0.00001)

# Since we may have gotten more than 1 ob in our time window, only keep the latest observation
data = df.groupby('station').tail(1)

Note that the station information variables (like longitude) have a different shape than the data variables (like temperature).

In [ ]:
print(data['tmpf'])
print(data['lon'])

Subset out global surface data

In [ ]:
loc_subset = ((df.lon <= -50) &
              (df.lon >= -180) &
              (df.lat >= 20) &
              (df.lat <= 70))
data = data[loc_subset]

Now we need to pull apart the data and perform some modifications, like converting winds to components and convert sky coverage percent to codes (octets) suitable for plotting.

In [ ]:
import numpy as np

from metpy.calc import wind_components, altimeter_to_sea_level_pressure
from metpy.plots import wx_code_to_numeric
from metpy.units import units

# Access via Pandas DataFrame and downcast to numpy array to attach units
lats = data.lat.values
lons = data.lon.values
tmpf = data.tmpf.values * units.degF
dwpf = data.dwpf.values * units.degF
alti = data.alti.values * units.inHg
elev = data.elevation.values * units.meter
stid = data.station.values

Generate MSLP values from the Altimeter reading. Utilize the MetPy function to compute with eleveation and temperature.

In [ ]:
mslp = altimeter_to_sea_level_pressure(alti, elev, tmpf).to('hPa')

Calculate wind components from wind speed (sknt) and wind direction (drct) values.

In [ ]:
# Convert wind to components
u, v = wind_components(data.sknt.values * units.knot, data.drct.values * units.degree)

Convert cloud cover codes to numeric okta values using a small definition.

In [ ]:
# Need to handle missing (NaN) and convert to proper code
def assign_cloud_cover(code):
    if 'OVC' in code:
        return 1
    elif 'BKN' in code:
        return 6
    elif 'SCT' in code:
        return 4
    elif 'FEW' in code:
        return 2
    else:
        return 0
    
skyc = data.skyc1.astype('str')
cloud_cover = skyc.apply(assign_cloud_cover)

Assign the numeric weather codes to the METAR current weather text

In [ ]:
wxsym = data.wxcodes
wx = wx_code_to_numeric(wxsym.fillna(''))

### Create the map using cartopy and MetPy!

One way to create station plots with MetPy is to create an instance of `StationPlot` and call various plot methods, like `plot_parameter`, to plot arrays of data at locations relative to the center point.

In addition to plotting values, `StationPlot` has support for plotting text strings, symbols, and plotting values using custom formatting.

Plotting symbols involves mapping integer values to various custom font glyphs in our custom weather symbols font. MetPy provides mappings for converting WMO codes to their appropriate symbol. The `sky_cover` function below is one such mapping.

In [ ]:
%matplotlib inline

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from metpy.plots import StationPlot
from metpy.plots.wx_symbols import sky_cover, current_weather

# Set up a plot with map features
fig = plt.figure(figsize=(12, 12))
proj = ccrs.LambertConformal(central_longitude=-100, central_latitude=35)
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.set_extent([-124, -65, 20, 60], ccrs.PlateCarree())

ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black')
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='black')
ax.gridlines()

# Create a station plot pointing to an Axes to draw on as well as the location of points
stationplot = StationPlot(ax, lons, lats, transform=ccrs.PlateCarree(),
                          fontsize=10)
stationplot.plot_parameter('NW', tmpf, color='red')

# Add wind barbs
stationplot.plot_barb(u, v)

# Plot the sky cover symbols in the center. We give it the integer code values that
# should be plotted, as well as a mapping class that can convert the integer values
# to the appropriate font glyph.
stationplot.plot_symbol('C', cloud_cover, sky_cover)

plt.show()

Notice how there are so many overlapping stations? There's a utility in MetPy to help with that: `reduce_point_density`. This returns a mask we can apply to data to filter the points.

In [ ]:
from metpy.calc import reduce_point_density

# Project points so that we're filtering based on the way the stations are laid out on the map
proj = ccrs.Stereographic(central_longitude=-95, central_latitude=35)
xy = proj.transform_points(ccrs.PlateCarree(), lons, lats)

# Reduce point density so that there's only one point within a 200km circle
mask = reduce_point_density(xy, 200000)

Now we just plot with `arr[mask]` for every `arr` of data we use in plotting.

In [ ]:
# Set up a plot with map features
fig = plt.figure(figsize=(15, 15))
proj = ccrs.LambertConformal(central_longitude=-100, central_latitude=35)
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.set_extent([-124, -65, 20, 60], ccrs.PlateCarree())

ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black')
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='black')
ax.gridlines()

# Create a station plot pointing to an Axes to draw on as well as the location of points
stationplot = StationPlot(ax, lons[mask], lats[mask], transform=ccrs.PlateCarree(),
                          fontsize=10, clip_on=True)
stationplot.plot_parameter('NW', tmpf[mask], color='red')

stationplot.plot_barb(u[mask], v[mask])

stationplot.plot_symbol('C', cloud_cover[mask], sky_cover)
stationplot.plot_symbol('W', wx[mask], current_weather, color='blue')

ax.set_title(f'Surface Observations for {date} UTC')
plt.show()

More examples for MetPy Station Plots:
- [MetPy Examples](https://unidata.github.io/MetPy/examples/index.html)
- [MetPy Symbol list](https://unidata.github.io/MetPy/api/generated/metpy.plots.StationPlot.html#metpy.plots.StationPlot.plot_symbol)

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Activity: Modify our station plots!</p>
    To plot this image, we will use <b>matplotlib</b> to display our data. Let's first make sure we have the information we need: <br> 
<ul>
    <li>Modify the station plot (reproduced below) to include dewpoint, altimeter setting, as well as the station id. The station id can be added using the `plot_text` method on `StationPlot`.</li>
    <li>Re-mask the data to be a bit more finely spaced, say: 100 km</li>
    <li>Bonus Points: Use the `formatter` argument to `plot_parameter` to only plot the 3 significant digits of altimeter setting. (Tens, ones, tenths)</li>
</ul>
</div>

In [ ]:
# Use reduce_point_density

# Set up a plot with map features
fig = plt.figure(figsize=(12, 12))
proj = ccrs.LambertConformal(central_longitude=-100, central_latitude=35)
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.set_extent([-124, -65, 20, 60], ccrs.PlateCarree())

ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black')
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='black')
ax.gridlines()

# Create a station plot pointing to an Axes to draw on as well as the location of points

# Plot dewpoint

# Plot altimeter setting--formatter can take a function that formats values

# Plot station id

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="timeseries"></a>
## 3. Time Series request and plot
* Let's say we want the past days worth of data...
* ...for Denver (i.e. DVN)
* ...for the variables mean sea level pressure, air temperature, wind direction, and wind_speed

In [ ]:
# Set a date and time for the observations you want
date = datetime.utcnow()

# Set a time window to look for observations
dt = timedelta(hours=24)

# Use datetime and time window for previous 24 hours from chosen time
sdate = date - dt
edate = date

Generate data access URL with only selecting an individual station

In [ ]:
# Select Station
station = 'DEN'

# Construct the data access URL
# NOTE: Addition of station variable to only grab that single location
data_url = (f'http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={station}&'
            'data=all&tz=Etc/UTC&format=comma&latlon=yes&elev=yes&'
            f'year1={sdate.year}&month1={sdate.month}&day1={sdate.day}&hour1={sdate.hour}&minute1={sdate.minute}&'
            f'year2={edate.year}&month2={edate.month}&day2={edate.day}&hour2={edate.hour}&minute2={edate.minute}')

### Let's get the data!

Use Pandas remote csv reading capability to grab desired data

In [ ]:
# Make Missing values ('M') into NaNs
# Replace Trace precip with very small float value
# Use time as our index, parse dates to be datetime objects
df = pd.read_csv(data_url,
                 index_col='valid', parse_dates=['valid'],
                 skiprows=5, na_values=['M'], low_memory=False).replace('T', 0.00001)

Subset the data we got to only a few variables and drop times with missing values

In [ ]:
# Let's subset to only the values we want to work with
# 'alti', 'tmpf', 'sknt', 'drct'
data = df[['alti', 'tmpf', 'sknt', 'drct']].dropna()
print(data)

Let's get the time into a datetime object

In [ ]:
valid_times = data.index
print(valid_times)

### Now for the obligatory time series plot...

In [ ]:
wind_data = data.sknt

In [ ]:
from matplotlib.dates import DateFormatter, AutoDateLocator

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(valid_times, wind_data, color='tab:blue')

ax.set_title(f'Site: {station}     Most Recent Observation: {valid_times[-1].strftime("%Y/%m/%d %H:%M")} UTC')
ax.set_xlabel('Hour of day')
ax.set_ylabel('Wind Speed')
ax.grid(True)

# Improve on the default ticking
locator = AutoDateLocator()
hoursFmt = DateFormatter('%H')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(hoursFmt)

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Activity: Exploration</p>
    Tasks:<br> 
<ul>
    <li>Pick a different location</li>
    <li>Plot temperature and dewpoint together on the same plot</li>
</ul>
</div>

<a href="#top">Top</a>
<hr style="height:2px;">

## Additional Take Home Notes

### Decoding METAR

What if you find that you have undecoded METAR data?

MetPy has a METAR parser that we can see how it works using the raw metar data that we get with the remote data we accessed from the Iowa State archive.

In [ ]:
from metpy.io import parse_metar_to_dataframe

# Parse single METAR to a Pandas DataFrame
kden = parse_metar_to_dataframe(df.metar.iloc[0])

In [ ]:
from io import StringIO

from metpy.io import parse_metar_file

# Join all of the individual metars into a single string
str_obj = '\n'.join(val for val in df.metar)

# Use stringIO to create a file-like object to use with the
# parse_metar_file function from MetPy
obs = parse_metar_file(StringIO(str_obj))

In [ ]:
obs